In [1]:
import os

import pickle

import numpy as np
import pandas as pd

from collections import defaultdict


# 전체 테스트 데이터 가져와서 실험

In [2]:
from data_loaders.assist2009 import ASSIST2009
from torch.nn.functional import one_hot, binary_cross_entropy

dataset = ASSIST2009(100)

import json

with open("config.json") as f:
    config = json.load(f)
    model_config = config["dkt"]
    train_config = config["train_config"]
    
import torch 
from models.dkt import DKT

model = DKT(dataset.num_q, **model_config)
model.load_state_dict(torch.load('./ckpts/dkt/ASSIST2009/model.ckpt'))

model.eval()

DKT(
  (interaction_emb): Embedding(220, 100)
  (lstm_layer): LSTM(100, 100, batch_first=True)
  (out_layer): Linear(in_features=100, out_features=110, bias=True)
  (dropout_layer): Dropout(p=0.5, inplace=False)
)

# test dataset 정의
## 미리 저장해둔 test_indices 적용

In [3]:
import torch
from torch.utils.data import DataLoader, random_split

train_size = int(len(dataset) * 0.9)
test_size = len(dataset) - train_size

train_dataset, test_dataset = random_split(
        dataset, [train_size, test_size], generator=torch.Generator(device='cuda')
    )

In [4]:
if os.path.exists(os.path.join(dataset.dataset_dir, "train_indices.pkl")):
    with open(
        os.path.join(dataset.dataset_dir, "train_indices.pkl"), "rb"
    ) as f:
        train_dataset.indices = pickle.load(f)
    with open(
        os.path.join(dataset.dataset_dir, "test_indices.pkl"), "rb"
    ) as f:
        test_dataset.indices = pickle.load(f)

In [5]:
test_dataset.indices

[3033,
 2472,
 3162,
 2923,
 5573,
 2811,
 5855,
 1246,
 4762,
 2742,
 4477,
 5916,
 832,
 2451,
 426,
 235,
 4877,
 4876,
 2202,
 4117,
 3665,
 2136,
 6008,
 2680,
 2372,
 906,
 3528,
 4107,
 5005,
 741,
 4244,
 3516,
 4962,
 5760,
 3726,
 5110,
 1384,
 4599,
 5214,
 2099,
 4694,
 2637,
 5648,
 3508,
 3644,
 5515,
 576,
 758,
 3060,
 4398,
 1332,
 5278,
 3674,
 5058,
 5563,
 4297,
 2272,
 1784,
 1857,
 4137,
 5038,
 4106,
 2228,
 3131,
 1882,
 5591,
 4565,
 5165,
 6180,
 492,
 5413,
 3908,
 6097,
 2112,
 3303,
 3217,
 2198,
 5928,
 3489,
 4970,
 3209,
 74,
 18,
 508,
 4305,
 5604,
 4059,
 4406,
 2526,
 5596,
 5079,
 4921,
 932,
 228,
 524,
 2955,
 303,
 244,
 4826,
 3024,
 960,
 1577,
 5531,
 1538,
 2222,
 1827,
 407,
 315,
 5335,
 2006,
 5348,
 5404,
 4931,
 3623,
 2723,
 3864,
 2191,
 2251,
 6003,
 617,
 5190,
 3677,
 5284,
 6142,
 6111,
 1274,
 6200,
 5016,
 15,
 2268,
 1666,
 3576,
 5071,
 5584,
 3669,
 4952,
 1338,
 1925,
 3466,
 3490,
 375,
 2011,
 414,
 3314,
 2426,
 4141,
 350

In [6]:
from models.utils import collate_fn

# train_loader = DataLoader(
#         train_dataset, batch_size=256, shuffle=True,
#         collate_fn=collate_fn, generator=torch.Generator(device='cuda') #0607 hson
#     )
test_loader = DataLoader(
        test_dataset, batch_size=test_size, shuffle=False,
        collate_fn=collate_fn, generator=torch.Generator(device='cuda') #0607 hson
    )

# test dataloader INPUT 값 확인

In [7]:
for data in test_loader:
    q, r, qshft, rshft, m = data

In [8]:
data

(tensor([[  1.,   1.,   1.,  ...,  52.,   2.,   2.],
         [ 49.,  67.,  18.,  ...,  -0.,  -0.,  -0.],
         [ 58.,  58.,  59.,  ...,  -0.,  -0.,  -0.],
         ...,
         [  3.,   3.,   3.,  ...,  -0.,  -0.,  -0.],
         [ 67., 102., 102.,  ...,  -0.,  -0.,  -0.],
         [ 23.,  23.,  23.,  ..., 102., 102., 102.]]),
 tensor([[1., 0., 1.,  ..., 1., 0., 1.],
         [0., 0., 0.,  ..., -0., -0., -0.],
         [0., 0., 1.,  ..., -0., -0., -0.],
         ...,
         [0., 1., 1.,  ..., -0., -0., -0.],
         [0., 0., 1.,  ..., -0., -0., -0.],
         [0., 1., 1.,  ..., 0., 0., 1.]]),
 tensor([[  1.,   1.,   1.,  ...,   2.,   2.,   2.],
         [ 67.,  18.,  35.,  ...,  -0.,  -0.,  -0.],
         [ 58.,  59.,  58.,  ...,  -0.,  -0.,  -0.],
         ...,
         [  3.,   3.,   3.,  ...,  -0.,  -0.,  -0.],
         [102., 102., 102.,  ...,  -0.,  -0.,  -0.],
         [ 23.,  23.,  23.,  ..., 102., 102.,  35.]]),
 tensor([[0., 1., 1.,  ..., 0., 1., 1.],
         [0., 0.,

# 추가 필요 데이터 정의

In [9]:
DATASET_DIR = "datasets/ASSIST2009/"
with open(os.path.join(DATASET_DIR, "q2idx.pkl"), "rb") as f:
    q2idx = pickle.load(f)
idx2q = {v:k for k,v in q2idx.items()}

In [10]:
idx2q

{0: 'Absolute Value',
 1: 'Addition Whole Numbers',
 2: 'Addition and Subtraction Fractions',
 3: 'Addition and Subtraction Integers',
 4: 'Addition and Subtraction Positive Decimals',
 5: 'Algebraic Simplification',
 6: 'Algebraic Solving',
 7: 'Angles - Obtuse, Acute, and Right',
 8: 'Angles on Parallel Lines Cut by a Transversal',
 9: 'Area Circle',
 10: 'Area Irregular Figure',
 11: 'Area Parallelogram',
 12: 'Area Rectangle',
 13: 'Area Trapezoid',
 14: 'Area Triangle',
 15: 'Box and Whisker',
 16: 'Calculations with Similar Figures',
 17: 'Choose an Equation from Given Information',
 18: 'Circle Graph',
 19: 'Circumference ',
 20: 'Complementary and Supplementary Angles',
 21: 'Computation with Real Numbers',
 22: 'Congruence',
 23: 'Conversion of Fraction Decimals Percents',
 24: 'Counting Methods',
 25: 'D.4.8-understanding-concept-of-probabilities',
 26: 'Distributive Property',
 27: 'Divisibility Rules',
 28: 'Division Fractions',
 29: 'Effect of Changing Dimensions of a Shap

# idx 에 따른 결과값 테스트

In [61]:
def test(idx):
#     idx = 20


    outputs = model(q[idx].long(), r[idx].long())
    outputs = (outputs * one_hot(qshft[idx].long(), model.num_q)).sum(-1) 

    outputs = torch.masked_select(outputs, m[idx]).detach().cpu()
    t = torch.masked_select(rshft[idx], m[idx]).detach().cpu()


    idx_q = torch.masked_select(qshft[idx], m[idx]).detach().cpu()#.unique()
    idx_q = idx_q.tolist()


    student_q_seq = torch.masked_select(q[idx], m[idx]).detach().cpu().tolist()
    student_q_seq.append(idx_q[-1])
    print('\033[95m 학생의 문제 skill id 시퀀스 \033[0m: ', student_q_seq, '\n')

    student_r_seq = torch.masked_select(r[idx], m[idx]).detach().cpu().tolist()
    student_r_seq.append(t.tolist()[-1])
    print('\033[95m 각 문제 정오답 여부 시퀀스 \033[0m: ', student_r_seq, '\n')




    # print('qshft 문제 시퀀스: ', idx_q, '\n')

    print('output: ', outputs, '\n')

    print('true: ', t, '\n')



    try:
        roc_auc_s = metrics.roc_auc_score(y_true=t.numpy(), y_score=outputs.numpy())
    #     print(roc_auc_s)
    except:
        roc_auc_s = "all respond are same"
    #     print("all respond are same")
    print(roc_auc_s, '\n')




    r_dict = defaultdict(list)
    for idx, corr in enumerate(t.tolist()):
        r_dict[idx_q[idx]].append(corr)
    print('각 스킬별 정오답 시퀀스: ',r_dict, '\n')

    q_dict = defaultdict(list)
    for idx, p in enumerate(outputs.tolist()):
        q_dict[idx_q[idx]].append(p)
    print('확률 변화값: ',q_dict, '\n')

    q_last_dict = {}
    for k,v in q_dict.items():
        q_last_dict[k] = v[-1]
    print('스킬별 last 확률값: ',q_last_dict, '\n')

    q_mean_dict = {}
    for k,v in q_dict.items():
        q_mean_dict[k] = np.mean(v)
    print('스킬별 평균 확률값: ',q_mean_dict, '\n')




    lst = set(student_q_seq)
    skill_lst = {}
    for i in lst:
        skill_lst[int(i)]=idx2q.get(int(i))
    print(skill_lst, '\n')
    
    tmp_dict = {}
    print('스킬 별 다음 정오답 확률: \n')
    for k,v in q_dict.items():
        tmp_dict[skill_lst[k]] = v[-1]
        print(f"\033[95m {skill_lst[k]} \033[0m: {v[-1]}")
#     print('스킬별 확률값: ',tmp_dict, '\n')
    
    return 0

In [62]:
test(9)

 학생의 문제 skill id 시퀀스 :  [22.0, 83.0, 100.0, 82.0, 100.0, 100.0, 82.0, 82.0, 20.0] 

 각 문제 정오답 여부 시퀀스 :  [1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0] 

output:  tensor([0.2943, 0.3964, 0.4870, 0.5639, 0.6939, 0.4791, 0.5294, 0.7135],
       device='cpu') 

true:  tensor([0., 1., 0., 1., 1., 0., 1., 1.], device='cpu') 

all respond are same 

각 스킬별 정오답 시퀀스:  defaultdict(<class 'list'>, {83.0: [0.0], 100.0: [1.0, 1.0, 1.0], 82.0: [0.0, 0.0, 1.0], 20.0: [1.0]}) 

확률 변화값:  defaultdict(<class 'list'>, {83.0: [0.2943229675292969], 100.0: [0.39641210436820984, 0.5638629198074341, 0.6939181089401245], 82.0: [0.48700475692749023, 0.4790886640548706, 0.5294116735458374], 20.0: [0.7134891152381897]}) 

스킬별 last 확률값:  {83.0: 0.2943229675292969, 100.0: 0.6939181089401245, 82.0: 0.5294116735458374, 20.0: 0.7134891152381897} 

스킬별 평균 확률값:  {83.0: 0.2943229675292969, 100.0: 0.5513977110385895, 82.0: 0.4985016981760661, 20.0: 0.7134891152381897} 

{100: 'Translations', 82: 'Reflection', 83: 'Rotations'

0